In [6]:
import pandas as pd

from surprise import KNNWithZScore

from surprise import Dataset
from surprise import Reader

from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

from helpers import *

In [7]:
data = load_data("data/data_train.csv")
data.head()

,user,movie,rating
0,r44,c1,4
1,r61,c1,3
2,r67,c1,4
3,r72,c1,3
4,r86,c1,5


In [8]:
dataset = Dataset.load_from_df(data[['user', 'movie', 'rating']], Reader(rating_scale=(1, 5)))

In [12]:
param_grid = {'k': [10, 300], 'k_min': [1, 30]}
gs = GridSearchCV(KNNWithZScore, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs = -1, refit=True)
gs.fit(dataset)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

Computing the msd similarity matrix...
Done computing similarity matrix.
1.006993106484051
{'k': 300, 'k_min': 1}


In [14]:
algo = gs.best_estimator['rmse']
cross_validate(algo, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True, n_jobs=-1)

Evaluating RMSE, MAE of algorithm KNNWithZScore on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0037  1.0059  1.0051  1.0047  1.0059  1.0051  0.0008  
MAE (testset)     0.8141  0.8156  0.8145  0.8149  0.8151  0.8148  0.0005  
Fit time          64.25   77.33   91.92   106.33  116.60  91.29   18.94   
Test time         817.53  807.83  775.56  729.04  681.81  762.35  50.76   


{'test_rmse': array([1.00368412, 1.00591251, 1.00505964, 1.0046815 , 1.00592032]),
 'test_mae': array([0.81408437, 0.81557933, 0.8144548 , 0.81490164, 0.81510345]),
 'fit_time': (64.24701476097107,
  77.32929015159607,
  91.92268490791321,
  106.3316171169281,
  116.59701085090637),
 'test_time': (817.5262069702148,
  807.8317778110504,
  775.5592131614685,
  729.0404551029205,
  681.8089988231659)}

In [15]:
gen_submission(r'data/submission_knn_2.csv', algo)